In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Practice').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/19 22:40:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [39]:
df_pyspark = spark.read.csv("test2.csv", header=True, inferSchema=True)
# df_pyspark = spark.read.csv("test2.csv", header=True)

In [40]:
df_pyspark.show()

+------+----+----------+------+
|  Name| Age|Experience|Salary|
+------+----+----------+------+
|Anshul|  37|        15|250000|
| Rohan|  33|        11|450000|
|Poorva|  35|         4| 10000|
|Saloni|  31|         1|  9000|
|  null|null|         1|  8000|
|Ruchir|   5|         0|  null|
|Cheeku|   1|      null|     0|
+------+----+----------+------+



In [41]:
# drop columns
df_pyspark.drop('Name').show()

+----+----------+------+
| Age|Experience|Salary|
+----+----------+------+
|  37|        15|250000|
|  33|        11|450000|
|  35|         4| 10000|
|  31|         1|  9000|
|null|         1|  8000|
|   5|         0|  null|
|   1|      null|     0|
+----+----------+------+



In [42]:
df_pyspark.show()

+------+----+----------+------+
|  Name| Age|Experience|Salary|
+------+----+----------+------+
|Anshul|  37|        15|250000|
| Rohan|  33|        11|450000|
|Poorva|  35|         4| 10000|
|Saloni|  31|         1|  9000|
|  null|null|         1|  8000|
|Ruchir|   5|         0|  null|
|Cheeku|   1|      null|     0|
+------+----+----------+------+



In [43]:
# drop null values
df_pyspark.na.drop().show()

+------+---+----------+------+
|  Name|Age|Experience|Salary|
+------+---+----------+------+
|Anshul| 37|        15|250000|
| Rohan| 33|        11|450000|
|Poorva| 35|         4| 10000|
|Saloni| 31|         1|  9000|
+------+---+----------+------+



In [44]:
df_pyspark.na.drop(how='all').show()

+------+----+----------+------+
|  Name| Age|Experience|Salary|
+------+----+----------+------+
|Anshul|  37|        15|250000|
| Rohan|  33|        11|450000|
|Poorva|  35|         4| 10000|
|Saloni|  31|         1|  9000|
|  null|null|         1|  8000|
|Ruchir|   5|         0|  null|
|Cheeku|   1|      null|     0|
+------+----+----------+------+



In [45]:
## threshold
df_pyspark.na.drop(how='any', thresh=3).show()

+------+---+----------+------+
|  Name|Age|Experience|Salary|
+------+---+----------+------+
|Anshul| 37|        15|250000|
| Rohan| 33|        11|450000|
|Poorva| 35|         4| 10000|
|Saloni| 31|         1|  9000|
|Ruchir|  5|         0|  null|
|Cheeku|  1|      null|     0|
+------+---+----------+------+



In [46]:
## subset
df_pyspark.na.drop(how='any', subset=['Experience']).show()

+------+----+----------+------+
|  Name| Age|Experience|Salary|
+------+----+----------+------+
|Anshul|  37|        15|250000|
| Rohan|  33|        11|450000|
|Poorva|  35|         4| 10000|
|Saloni|  31|         1|  9000|
|  null|null|         1|  8000|
|Ruchir|   5|         0|  null|
+------+----+----------+------+



In [47]:
# fill
df_pyspark.na.fill("missing").show()

+-------+----+----------+------+
|   Name| Age|Experience|Salary|
+-------+----+----------+------+
| Anshul|  37|        15|250000|
|  Rohan|  33|        11|450000|
| Poorva|  35|         4| 10000|
| Saloni|  31|         1|  9000|
|missing|null|         1|  8000|
| Ruchir|   5|         0|  null|
| Cheeku|   1|      null|     0|
+-------+----+----------+------+



In [48]:
df_pyspark.na.fill("missing", ["Experience", 'Salary']).show()

+------+----+----------+------+
|  Name| Age|Experience|Salary|
+------+----+----------+------+
|Anshul|  37|        15|250000|
| Rohan|  33|        11|450000|
|Poorva|  35|         4| 10000|
|Saloni|  31|         1|  9000|
|  null|null|         1|  8000|
|Ruchir|   5|         0|  null|
|Cheeku|   1|      null|     0|
+------+----+----------+------+



In [49]:
# replace missing with mean/median values
from pyspark.ml.feature import Imputer

col_list = ['Age', 'Experience', 'Salary']

imputer = Imputer(
    inputCols = col_list,
    outputCols = ["{}_imputed".format(c) for c in col_list]
).setStrategy("mean")

In [50]:
imputer.fit(df_pyspark).transform(df_pyspark).show()

+------+----+----------+------+-----------+------------------+--------------+
|  Name| Age|Experience|Salary|Age_imputed|Experience_imputed|Salary_imputed|
+------+----+----------+------+-----------+------------------+--------------+
|Anshul|  37|        15|250000|         37|                15|        250000|
| Rohan|  33|        11|450000|         33|                11|        450000|
|Poorva|  35|         4| 10000|         35|                 4|         10000|
|Saloni|  31|         1|  9000|         31|                 1|          9000|
|  null|null|         1|  8000|         23|                 1|          8000|
|Ruchir|   5|         0|  null|          5|                 0|        121166|
|Cheeku|   1|      null|     0|          1|                 5|             0|
+------+----+----------+------+-----------+------------------+--------------+



In [51]:
imputer = Imputer(
    inputCols = col_list,
    outputCols = ["{}_imputed".format(c) for c in col_list]
).setStrategy("median")

In [52]:
imputer.fit(df_pyspark).transform(df_pyspark).show()

+------+----+----------+------+-----------+------------------+--------------+
|  Name| Age|Experience|Salary|Age_imputed|Experience_imputed|Salary_imputed|
+------+----+----------+------+-----------+------------------+--------------+
|Anshul|  37|        15|250000|         37|                15|        250000|
| Rohan|  33|        11|450000|         33|                11|        450000|
|Poorva|  35|         4| 10000|         35|                 4|         10000|
|Saloni|  31|         1|  9000|         31|                 1|          9000|
|  null|null|         1|  8000|         31|                 1|          8000|
|Ruchir|   5|         0|  null|          5|                 0|          9000|
|Cheeku|   1|      null|     0|          1|                 1|             0|
+------+----+----------+------+-----------+------------------+--------------+

